# Binary Spam Classifier
The purpose of this notebook is to train scikit-learn binary spam classification models on the full corpus for deployment.


In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.sparse import issparse
from sklearn.pipeline import Pipeline


from dotenv import load_dotenv
import os
import mlflow
import mlflow.sklearn
import joblib
import time

import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
import string

In [2]:
RANDOM_STATE = 123

## Import Data

In [3]:
DATASET_FILENAME = "dataset-v2.pkl"
DATA_PATH = os.path.join("..", "data", DATASET_FILENAME)

In [4]:
df = pd.read_pickle(DATA_PATH)
df.head()

,id,url,author,title,body,labels
0,2172615896,https://api.github.com/repos/lindsaypj/portfol...,lindsaypj,Clicking accordion topic updates terminal text...,,"[bug, pending MERGE]"
1,2320286757,https://api.github.com/repos/JosefNemec/Playni...,fm-117,[IGDB Metadata] Some games cannot be found,When looking for metadata using the IGDB searc...,[bug]
2,2242364830,https://api.github.com/repos/nomicflux/scrpg-h...,nomicflux,EnergyChoice needs to handle Physical for Powe...,,"[bug, enhancement]"
3,2309230756,https://api.github.com/repos/fluentcms/FluentC...,pournasserian,Why should we have `StopPropagation` property ...,,"[enhancement, question]"
4,2229948881,https://api.github.com/repos/hpsaturn/ESPNowCa...,beniroquai,Many-to-one connection,"Hey @hpsaturn, I stumbled upon your project wh...","[enhancement, question]"


## Preprocess data
- Split the input features (issue content) and target feature (label)
- For binary classification, convert all non-"spam" issues into "not-spam"

In [5]:
X = df['title'] + ' ' + df['body']
y = df['labels'].apply(lambda x: 'spam' if 'spam' in x else 'not-spam')

## Config MLflow

In [6]:
EXPERIMENT_NAME = "binary_classifier_corpus_2"
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

EXPERIMENT_ID = experiment.experiment_id
RUN_UID = str(int(time.time()))

print("Experiment ID:",EXPERIMENT_ID)
print("Experiment Name:", EXPERIMENT_NAME)


Experiment ID: 2
Experiment Name: binary_classifier_corpus_2


## Define Models
- Use statistical models and a simple neural network for variety
- Notice that we are not using SVC due to computational cost
- Save the models in a dictionary with their corresponding hyperparameters for easier indexing

In [7]:
logreg_hyperparameters = {
    'penalty': 'l2',
    'solver': 'lbfgs',
    'random_state': RANDOM_STATE
}
nn_hyperparameters = {
    'learning_rate': 'adaptive',
    'alpha': 0.0001,
    'max_iter': 1000,
    'activation': 'relu',
    'solver': 'adam',
    'random_state': RANDOM_STATE
}
rf_hyperparameters = {
    'max_depth': None,
    'n_estimators': 100, 
    'max_features': 'sqrt',
    'random_state': RANDOM_STATE
}

logreg = LogisticRegression(**logreg_hyperparameters)
nb = GaussianNB()
knn = KNeighborsClassifier(n_neighbors=5)
nn = MLPClassifier(**nn_hyperparameters)
rf = RandomForestClassifier(**rf_hyperparameters)

models = {"logreg": logreg, "nb": nb, "knn": knn, "nn": nn, "rf": rf}

## Define Vectorizers
- Text input must be converted into numerical input (vectors) before classification
- For computational efficiency, we'll use non-neural vectorizers Bag-of-Words and TFIDF

In [8]:
STOP_WORDS = set(stopwords.words('english') + list(string.punctuation))
STOP_WORDS = list(STOP_WORDS)

vectorizer_params = {
    'ngram_range': (1,3),
    'lowercase': True,
    'min_df': 5,
    'max_features': 100000
}
count_vectorizer = CountVectorizer(stop_words=STOP_WORDS, **vectorizer_params) # Bag-of-Words vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=STOP_WORDS, **vectorizer_params)

vectorizers = {"count_vectorizer": count_vectorizer, "tfidf_vectorizer": tfidf_vectorizer}

## Train Model On Full Dataset
- Select the best performing classifier-vectorizer model pair from the experiments
- Save as pipeline in MLflow artifacts folder

In [10]:
def train_and_log_model(X, y, model, model_name, model_params, vectorizer, vectorizer_name):
    registered_model_name = f"{model_name}_{vectorizer_name}"
    run_name = f"{registered_model_name}_{RUN_UID}"

    pipeline = Pipeline([
        ('vectorizer', vectorizer),
        ('model', model)
    ])

    pipeline.fit(X, y)
    run_id = None

    with mlflow.start_run(experiment_id=EXPERIMENT_ID, run_name=run_name) as run:
        # Log model
        mlflow.sklearn.log_model(pipeline, model_name, registered_model_name=registered_model_name)

        # Log model parameters
        model_params = model.get_params()
        mlflow.log_params(model_params)
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("vectorizer_name", vectorizer_name)

        run_id = run.info.run_id

    mlflow.end_run()
    return run_id

In [11]:
model_params = {
    'model': nn,
    'model_name': 'nn',
    'model_params': nn_hyperparameters
}
vectorizer_params = {
    'vectorizer': tfidf_vectorizer,
    'vectorizer_name': 'tfidf_vectorizer'
}
run_id = train_and_log_model(X, y, **model_params, **vectorizer_params)

2024/06/19 12:51:56 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

Registered model 'nn_tfidf_vectorizer' already exists. Creating a new version of this model...
2024/06/1

## Import and run the classifier
- Import the binary classifier as a pickle file from mlartifacts
- Run predictions using `.predict()`

In [12]:
model_uri = os.path.join('mlartifacts', EXPERIMENT_ID, run_id, 'artifacts', model_params['model_name'], 'model.pkl')
print(model_uri)
binary_classifier = joblib.load(model_uri)

mlartifacts\2\0579ea92a6c7494e9bfdf42813fe3867\artifacts\nn\model.pkl


In [21]:
SAMPLE_SPAM_TEXT = "please fix"
spam_prediction = binary_classifier.predict([SAMPLE_SPAM_TEXT])[0]
spam_prediction

'not-spam'